1. Activate the virtual environment: 
source /Users/sofiamorenolasa/Desktop/TFG/.venv/bin/activate
2. Switch to the virtual environment

In [17]:
import pandas as pd

file_path = "../data/PARAMETROS_TJ2.csv"

# Use latin1 encoding
data = pd.read_csv(file_path, delimiter=";", encoding="latin1")

# Display the first few rows
print(data.head())

   N_DESCARGA       fecha   hora comentarioDesc comentarioExp configuracion  \
0         112  19971217.0  19:05            NaN           NaN           NaN   
1         113  19971217.0  19:09            NaN           NaN           NaN   
2         114  19971217.0  19:55            NaN           NaN           NaN   
3         115  19971218.0  11:08            NaN           NaN           NaN   
4         116  19971218.0  11:28            NaN           NaN           NaN   

   potencia_radiada  energia_diamagnetica  retraso_densidad_girotron  zeff  \
0               NaN                   NaN                        NaN   NaN   
1               NaN                   NaN                        NaN   NaN   
2               NaN                   NaN                        NaN   NaN   
3               NaN                   NaN                        NaN   NaN   
4               NaN                   NaN                        NaN   NaN   

   ...  IAccel_nominal_NBI2  tini_NBI2  longitud_pulso_n

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/2676448733.py:6: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=";", encoding="latin1")


Next, we sort the PARAMETROS_TJ2.csv file by the N_DESCARGA field and save the sorted data to a new file named PARAMETROS_TJ2_ORDENADOS.csv. This ensures the rows are ordered starting from the smallest N_DESCARGA value to the largest.

In [14]:
import pandas as pd

# Load the dataset
input_file_path = "../data/PARAMETROS_TJ2.csv"  # Replace with the correct path
output_file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"

# Read the data
data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")

# Convert N_DESCARGA to numeric, coercing errors to NaN
data['N_DESCARGA'] = pd.to_numeric(data['N_DESCARGA'], errors='coerce')

# Drop rows where N_DESCARGA is NaN (invalid values)
data = data.dropna(subset=['N_DESCARGA'])

# Ensure N_DESCARGA is an integer
data['N_DESCARGA'] = data['N_DESCARGA'].astype(int)

# Sort by N_DESCARGA in ascending order
data_sorted = data.sort_values(by='N_DESCARGA')

# Save the sorted dataset to a new CSV file
data_sorted.to_csv(output_file_path, index=False, sep=";", encoding="latin1")

print(f"Sorted data saved to {output_file_path}")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/1307042932.py:8: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")


Sorted data saved to ../data/PARAMETROS_TJ2_ORDENADOS.csv


Next, we will convert this .csv file to a .txt to help with training with a RAG system

MODIFY THE TEXT BELOW TO INCLUDE ONLY THE IMPORTANT FIELDS.

In [24]:
import os
import pandas as pd
import re

def process_csv_to_txt(csv_path, txt_path):
    """
    Processes a CSV file with semicolon delimiters, extracts and cleans relevant data,
    and saves it as a .txt file.

    Args:
        csv_path (str): Path to the input CSV file.
        txt_path (str): Path to save the output .txt file.
    """
    try:
        # Read the CSV file with semicolon as the delimiter
        data = pd.read_csv(csv_path, delimiter=';', encoding='latin1', on_bad_lines='skip')
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return

    # Initialize the text content
    text = ''

    # Iterate over the rows of the CSV
    for index, row in data.iterrows():
        # Extract the necessary fields (customize this based on your analysis needs)
        descarga = row.get('N_DESCARGA', 'N/A')
        fecha = row.get('fecha', 'N/A')
        comentario = row.get('comentarioDesc', 'N/A')
        configuracion = row.get('configuracion', 'N/A')
        energia_diamagnetica = row.get('energia_diamagnetica', 'N/A')

        # Format the extracted data into a readable string
        text += (
            f"Descarga: {descarga}\n"
            f"Fecha: {fecha}\n"
            f"Comentario: {comentario}\n"
            f"Configuración: {configuracion}\n"
            f"Energía Diamagnética: {energia_diamagnetica}\n"
            f"{'-'*40}\n"  # Separator for readability
        )

    # Save the processed text to the output file
    try:
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
        print(f"Processed data saved to {txt_path}")
    except Exception as e:
        print(f"Error saving to {txt_path}: {e}")


# Input and output paths
csv_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"
txt_folder = "../data/txt_output/"

# Ensure the output folder exists
os.makedirs(txt_folder, exist_ok=True)

# Generate the output .txt file name
txt_path = os.path.join(txt_folder, "PARAMETROS_TJ2_ORDENADOS.txt")

# Process the CSV and save it as a .txt file
process_csv_to_txt(csv_path, txt_path)

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/1245340582.py:16: DtypeWarning: Columns (52,54,55,96,105,108,110,122,125,129,132,133,137,138,143,144,145,150,151,154) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_path, delimiter=';', encoding='latin1', on_bad_lines='skip')


Processed data saved to ../data/txt_output/PARAMETROS_TJ2_ORDENADOS.txt


Hay que borrar los datos repetidos, hay muchos donde la hora y la fecha son iguales, aunque el numero de descarga sea diferente, así que habría que dejar solo uno. 

Ahora hay que crear una base de datos vectorial a partir de los archivos .txt (database_creator)